In [1]:
from astropy.table import Table
from lens import Lens
from astropy.io import fits
import sqlutil as sqlutil 
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
import lensutils as util
import numpy as np

In [2]:
TGASLensId = 5332606518269523072
SourcePPMXLid = 6197608744359075309
GaiaSourceId = 5332606346467258496

querystringAPASS = 'select vjmag,bjmag,gmag,rmag, imag from gaia_dr1_aux.gaia_source_apass_xm where source_id=' + str(TGASLensId)
querystringTGAS = 'select ra, dec, source_id,pmra,pmdec,ref_epoch,phot_g_mean_mag,parallax,ra,dec from gaia_dr1.tgas_source where source_id=' + str(TGASLensId)
querystringPPMXL ='select ra, dec, id,Kmag, Jmag,pmra,pmdec from ppmxl.main where id=' + str(SourcePPMXLid)
querystringGAIASource = 'select ra, dec, source_id,pmra,pmdec,ref_epoch,phot_g_mean_mag,parallax from gaia_dr1.gaia_source where source_id=' + str(GaiaSourceId)
queryStringTGASS2mass = 'select j_m,h_m,k_m from gaia_dr1_aux.gaia_source_2mass_xm where source_id=' + str(TGASLensId)
queryStringTGASSDSS = 'select cmodelmag_u, cmodelmag_g ,cmodelmag_r, cmodelmag_i, cmodelmag_z from gaia_dr1_aux.gaia_source_sdssdr9_xm where source_id=' + str(TGASLensId)
source = Table.read('../Desktop/tgas_dist_all_v01.csv',format='csv')
sourcemask = source['3:SourceId'] == TGASLensId
source = source[sourcemask]
lensDist = source['19:rMo[pc]'][0]

In [3]:
lensRa, lensDec, id,pmra,pmdec,ref_epoch,gMag,parallax,ra,dec = sqlutil.get(querystringTGAS,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('Lens Gmag,pmra,pmdec,lensDist,ra,dec:' + str(gMag) + str(pmra) + str(pmdec)+str(parallax)+str(lensDist)+str(ra)+str(dec))

sourceRa,sourceDec,sourceId,kmag,jmag,spmra,spmdec = sqlutil.get(querystringPPMXL,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('PPMXL Source Ra,Dec,kmag,Jmag,pmra,pmdec,parallax: '+str(sourceRa)+str(sourceDec)+str(kmag)+str(jmag)+str(spmra)+str(spmdec))


GsourceRa, GsourceDec, id,sourcepmra,sourcepmdec,ref_epoch,sourcegMag,parallax = sqlutil.get(querystringGAIASource,db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('GAIA Source Ra,Dec,Gmag,Parallax: '+str(GsourceRa)+str(GsourceDec)+str(sourcegMag)+str(sourcepmra)+str(sourcepmdec)+str(parallax))

lensj , lensh, lensk = sqlutil.get(queryStringTGASS2mass,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('TGAS lens 2mass bands j h k: ' + str(lensj) + str(lensh) + str(lensk)) 

lensu , lensg, lensr, lensi,lensz = sqlutil.get(queryStringTGASSDSS,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('TGAS lens SDSS bands ugriz: ' + str(lensu) + str(lensg) + str(lensr) + str(lensi)+str(lensz)) 

vjmag,bjmag,gmag,rmag, imag = sqlutil.get(querystringAPASS,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')
print('TGAS lens apass bands vj,bj,g,r: ' + str(vjmag) + str(bjmag)+str(gmag)+str(rmag))


Lens Gmag,pmra,pmdec,lensDist,ra,dec:[ 11.41018521][ 2662.03572627][-345.18255501][ 215.7823335]4.634313168[ 176.4549073][-64.84295714]
PPMXL Source Ra,Dec,kmag,Jmag,pmra,pmdec,parallax: [ 176.463694][-64.843343][ 15.22099972][ 16.16300011][-19.5][-17.89999962]
GAIA Source Ra,Dec,Gmag,Parallax: [ 176.46360456][-64.84329779][ 18.46490291][ nan][ nan][ nan]
TGAS lens 2mass bands j h k: [ 15.87899971][ 15.38599968][ 15.11900043]
TGAS lens SDSS bands ugriz: [][][][][]
TGAS lens apass bands vj,bj,g,r: [][][][]


In [19]:
util.calc_centroid_shift(0.1,lensDist,18.97)

13.2205590499
1.43488637117


4.6736813298180921

In [21]:

util.calc_centroid_shift(0.5,lensDist,18.97)


29.5620687362
0.641700693185


7.8655605956185477

In [23]:
util.calc_centroid_shift(1.0,lensDist,18.97)


41.8070785385
0.453750911643


8.599703950578002

In [15]:
util.calc_centroid_shift(1.0,lensDist,0.97)

0.48486949153218706

In [14]:
goodlens = Lens(5332606518269523072,176.4549073,-64.84295714,2662.03572627,-345.18255501,2015.0)

In [15]:
time = goodlens.get_time_closest_app_sourcepm(176.46360456,-64.84329779,-19.5,-17.89999962)
dist = goodlens.get_dist_closest_app_sourcepm(176.46360456,-64.84329779,-19.5,-17.89999962)

In [18]:
print(time)
print(dist)

2018.68950929
18.9680317634


In [12]:
TGASLensId = 5332606518269523072
querystringTGAS2massLens = 'select j_m,k_m,parallax,phot_g_mean_mag from gaia_dr1_aux.gaia_source_2mass_xm where source_id=' + str(TGASLensId)

JmagLens,kMagLens,parallaxLens,gMagLens = sqlutil.get(querystringTGAS2massLens,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

In [15]:
absMagLens = gMagLens + 5 * (np.log10(parallaxLens / 1000.0) + 1)
colorLens = JmagLens - kMagLens

In [16]:
print(absMagLens)
print(colorLens)

[ 13.08026463]
[ 0.75999928]


In [9]:
querystringTGAS2mass= 'select j_m,k_m,parallax,phot_g_mean_mag from gaia_dr1_aux.gaia_source_2mass_xm where parallax > 0.0001 and j_m > 0.0001 and k_m > 0.0001 limit 100000'

Jmag,kMag,parallax,gMag = sqlutil.get(querystringTGAS2mass,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

color = Jmag-kMag
#nanmask = np.isnan(parallax)

#colorf = color[~nanmask]
#parallaxf = parallax[~nanmask]
#gMagf = gMag[~nanmask]

absmag = gMag + 5 * (np.log10(parallax / 1000.0) + 1)

print(color)

[ 0.37699986  0.70699978  0.7329998  ...,  0.29100037  0.33500004
  0.33600044]


In [8]:
print(Jmag-kMag)

[ 0.75800037  0.63899994  0.53700066 ...,  1.03299999  1.43899918
  1.08199978]


In [2]:
util.lens_on_color_mag(5332606518269523072,'test.png')